In [13]:
import ee

In [14]:
ee.Authenticate()

True

In [15]:
ee.Initialize()

In [18]:
aoi = ee.Geometry.Rectangle([105.0, 10.0, 106.0, 11.0])  # Ví dụ tọa độ TP HCM

# Lọc bộ sưu tập Landsat 8 Surface Reflectance (Landsat 8 SR)
landsat_collection = ee.ImageCollection("LANDSAT/LE07/C02/T1") \
    .filterBounds(aoi) \
    .filterDate('2019-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20))  # Lọc theo mức độ che phủ mây dưới 20%

# Lấy hình ảnh đầu tiên trong bộ sưu tập
landsat_image = landsat_collection.first()

# Xem thông tin về ảnh
info = landsat_image.getInfo()
print(info)

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [7981, 6901], 'crs': 'EPSG:32648', 'crs_transform': [30, 0, 607485, 0, -30, 1384515]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [7981, 6901], 'crs': 'EPSG:32648', 'crs_transform': [30, 0, 607485, 0, -30, 1384515]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [7981, 6901], 'crs': 'EPSG:32648', 'crs_transform': [30, 0, 607485, 0, -30, 1384515]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [7981, 6901], 'crs': 'EPSG:32648', 'crs_transform': [30, 0, 607485, 0, -30, 1384515]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [7981, 6901], 'crs': 'EPSG:32648', 'crs_transform': [30, 0, 607485, 0, -30, 1384515]}, {'id': 'B6_V

In [19]:
landsat_image

In [20]:
import folium

# Lấy URL hình ảnh từ GEE
url = landsat_image.getThumbURL({
    'bands': ['B4', 'B3', 'B2'],  # Chọn các dải phổ tương ứng với Red, Green, Blue
    'min': 0,
    'max': 3000,
    'dimensions': 512
})

# Tạo bản đồ folium
map = folium.Map(location=[10.5, 105.5], zoom_start=10)
folium.raster_layers.ImageOverlay(
    image=url,
    bounds=[[10.0, 105.0], [11.0, 106.0]],
    opacity=0.7
).add_to(map)

# Hiển thị bản đồ
map.save('landsat_image.html')

In [26]:
def buildSRCollection(startYear, endYear, startDay, endDay, aoi, maskThese=['cloud', 'shadow', 'snow']):
    """
    Build a collection of surface reflectance images after applying cloud and shadow masking.
    """
    # Define the date range based on the input
    start_date = ee.Date.fromYMD(startYear, int(startDay.split('-')[0]), int(startDay.split('-')[1]))
    end_date = ee.Date.fromYMD(endYear, int(endDay.split('-')[0]), int(endDay.split('-')[1]))
    
    # Load Landsat surface reflectance image collection
    collection = ee.ImageCollection("LANDSAT/LE07/C02/T1") \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20))  # Filter images with less than 20% cloud cover
    
    # Function to mask clouds, shadows, and snow
    def maskLandsatSR(image):
        qa = image.select('QA_PIXEL')

        # Cloud mask
        cloud_bit = 1 << 3
        mask_cloud = qa.bitwiseAnd(cloud_bit).eq(0)
        
        # Shadow mask
        shadow_bit = 1 << 4
        mask_shadow = qa.bitwiseAnd(shadow_bit).eq(0)
        
        # Snow mask
        snow_bit = 1 << 5
        mask_snow = qa.bitwiseAnd(snow_bit).eq(0)
        
        # Combine the masks
        combined_mask = mask_cloud.And(mask_shadow).And(mask_snow)
        return image.updateMask(combined_mask)
    
    # Apply the masking function to the collection (server-side mapping)
    # Tạo danh sách trống để lưu các ảnh sau khi áp dụng mask
    masked_images = []

    # Lặp qua từng ảnh trong ImageCollection và áp dụng hàm maskLandsatSR
    for image in collection.toList(collection.size()).getInfo():
        img = ee.Image(image['id'])  # Lấy id của ảnh từ image
        masked_img = maskLandsatSR(img)  # Áp dụng hàm mask
        masked_images.append(masked_img)  # Thêm ảnh sau khi mask vào danh sách

    # Chuyển đổi danh sách các ảnh thành một ImageCollection
    masked_collection = ee.ImageCollection(masked_images)
    
    # Function to calculate medoid composite for a specific year
    def medoid_composite(year):
        start = ee.Date.fromYMD(year, int(startDay.split('-')[0]), int(startDay.split('-')[1]))
        end = ee.Date.fromYMD(year, int(endDay.split('-')[0]), int(endDay.split('-')[1]))
        
        # Lọc bộ sưu tập theo năm
        filtered_collection = masked_collection.filterDate(start, end)
        
        # Tính ảnh trung bình để so sánh
        median_image = filtered_collection.median()

        # Hàm tính toán khoảng cách Euclidean từ ảnh đến ảnh trung vị
        def calculate_distance(image):
            diff = image.subtract(median_image).pow(ee.Image.constant(2))  # (Ảnh - trung vị)^2
            distance = diff.reduce('sum').sqrt()  # Tính tổng các băng và căn bậc hai
            return distance.addBands(image)  # Thêm ảnh ban đầu làm băng
        
        # Áp dụng hàm tính khoảng cách lên bộ sưu tập
        distance_collection = filtered_collection.map(calculate_distance)
        
        # Chọn ảnh có khoảng cách nhỏ nhất đến ảnh trung vị
        medoid = distance_collection.reduce(ee.Reducer.min(2)).select([1, 2, 3, 4, 5, 6], ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
        
        return medoid.set('year', year)
    
    # Create a list of years and map over them using Earth Engine's server-side .map()
    years = ee.List.sequence(startYear, endYear)
    
    def createMedoidImage(year):
        return medoid_composite(ee.Number(year))
    
    # Map over the years to create a collection of medoid composites
    medoid_images = ee.ImageCollection(years.map(createMedoidImage))
    
    return medoid_images


# Example usage of the function (this would execute in the Earth Engine cloud environment)
aoi = ee.Geometry.Point([-122.8848, 43.7929])  # Define an area of interest (AOI)
startYear = 2015
endYear = 2020
startDay = '06-01'
endDay = '09-30'

# Run the function
sr_collection = buildSRCollection(startYear, endYear, startDay, endDay, aoi)

EEException: A mapped function's arguments cannot be used in client-side operations